In [4]:
import sklearn as sk
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import model_selection
from sklearn import cross_validation
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import scoring
import pydotplus
import time
from IPython.display import Image  
import pydot 
import warnings
warnings.filterwarnings("ignore")

In [7]:
start_time = time.time()
df = pd.read_csv('C:\\users\\user\\cc_sample.txt', sep=';',index_col=False, decimal=',') 
print(-start_time + time.time()," sec")

2.7220051288604736  sec


In [8]:
start_time = time.time()
#Формируем сэмпл для разработки
df_train, df_test = train_test_split(df,random_state=1)
df_train = df_train.reset_index()
df_test = df_test.reset_index()
x_sample_train = df_train.copy()
x_sample_test = df_test.copy()
x_sample_train = x_sample_train.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90','index'], axis=1)
x_sample_test = x_sample_test.drop(['CONTRACT_SRC_CODE','SCORE_FINAL','BAD_12_FLAG90','index'], axis=1)
#test_col_list = ['RATE_TR_ALL_L3_6M', 'SD_GENDER_CD']
#x_sample_train = x_sample_train[test_col_list]
#x_sample_test = x_sample_test[test_col_list]
#Целевая переменная
y_train = df_train["BAD_12_FLAG90"][df_train['BAD_12_FLAG90'].notnull()] 
y_test = df_test["BAD_12_FLAG90"][df_test['BAD_12_FLAG90'].notnull()] 
#Процедура отбора переменных по критерию порогового IV
var_list_bins = iv_selection(x_sample_train,y_train,0.02)
var_list = list(var_list_bins.keys())
x_sample_train = x_sample_train[var_list]
x_sample_test = x_sample_test[var_list]
print('________________________________________________________________________________________________________________')
#Выводим графики WOE по переменным
#for col in x_sample.columns: print_woe_graph(iv_table(binning(x_sample[col],y,max_bins=10,mode='normal'),y),col)
#print('________________________________________________________________________________________________________________')
#Преобразуем в one-hot
x_dev_train = dev_to_one_hot(x_sample_train,y_train,list_optimal_bins = var_list_bins)
x_dev_test = dev_to_one_hot(x_sample_test,y_test,list_optimal_bins = var_list_bins)
#Исключаем коррелирующие переменные
x_dev_train = exclude_corr_factors(x_dev_train, 0.8, mode='var')
x_dev_train = x_dev_train[exclude_corr_factors(x_dev_train, 0.8, mode='list')]
x_dev_test = x_dev_test[exclude_corr_factors(x_dev_train, 0.8, mode='list')]
print('________________________________________________________________________________________________________________')
#Строим логит регрессию
model_logit = build_model(x_dev_train,y_train)
#Выводим визуально получившуюся скоркарту
col_list = x_dev_train.columns
scorecard = scorecard_view(col_list,model_logit,odds_X_to_one=100,odds_score=700,double_odds=25)
print(scorecard)
print('________________________________________________________________________________________________________________')
#Выводим GINI скоркарты
print('Development sample: ')
gini(model_logit,x_dev_train,y_train)
print('Validation sample: ')
gini(model_logit,x_dev_test,y_test)
print(-start_time + time.time()," sec")
start_time=0


Choosing variables with IV >  0
________________________________________________________________________________________________________________
                                                             
AVG_TERM_FACT   IV =  0.07753551948574
                    x   count  bad_rate  total_goods  total_bads  cumm_bads  \
0    001: (-1.0, 3.5]    7204  0.029983         6988         216        216   
1        002: MISSING  285686  0.041714       273769       11917      12133   
2     003: (3.5, 7.5]   27009  0.027398        26269         740      12873   
3    004: (7.5, 13.5]   52019  0.023472        50798        1221      14094   
4   005: (13.5, 46.5]   91693  0.021910        89684        2009      16103   
5   006: (46.5, 56.5]    4811  0.030555         4664         147      16250   
6  007: (56.5, 170.0]    4858  0.034788         4689         169      16419   

   cumm_goods  cumm_total   per_bad  per_good       woe        iv  
0        6988        7204  0.013155  0.015296  0.1507

KeyboardInterrupt: 